In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
import pandas as pd
import seaborn as sns
import anndata
import scanpy as sc
import genetools
from covid_serology import config

# Load data

In [2]:
vaccine_participants = pd.read_csv(
    f"{config.paths.data_dir}/pfizer_demographics.csv"
).dropna(how="all")
vaccine_participants

,PID,COVID Positive Ever?
0,pfizer00,No
1,pfizer01,No
2,pfizer02,No
3,pfizer03,No
4,pfizer04,No
5,pfizer05,No
6,pfizer06,No
7,pfizer07,No
8,pfizer08,No
9,pfizer09,No


In [3]:
vaccine_df = pd.read_csv(f"{config.paths.data_dir}/pfizer_data_final_ed_final.csv")
vaccine_df

,study_id,tp,plate,CoV2_S_IgG,CoV2_S_IgG.1,CoV2_S_IgG_Mean,NL63_S_IgG,NL63_S_IgG.1,NL63_S_IgG_Mean,CoV2_N_IgG,...,P9_RBD_ACE_B.1.429_percent,P9_RBD_ACE_B.1.351_percent,P9_RBD_ACE_B.1.243_percent,P9_RBD_ACE_P.1_percent,P9_RBD_ACE_B.1.526.2_percent,P9_RBD_ACE_B.1.1.7_percent,P9_RBD_ACE_B.1.1.7_E484_percent,P9_RBD_ACE_B.1.617_percent,P9_RBD_ACE_B.1.214_percent,P9_CoV2 _RBD_ACE_percent
0,pfizer00,D0,1,698.328,667.418,682.8730,6913.225,6517.880,6715.5525,881.258,...,13.725107,-77.090366,-11.747820,-4.809098,2.388070,4.780092,-4.851285,-3.914414,-3.179730,12.738288
1,pfizer01,D0,1,34.175,34.416,34.2955,707.973,694.801,701.3870,109.893,...,9.695624,-57.853730,-22.009318,-6.278746,0.269049,2.957650,-10.803043,-9.749725,-6.754975,0.127979
2,pfizer02,D0,1,55.502,59.408,57.4550,1475.624,1483.167,1479.3955,92.850,...,6.906935,-90.757272,-22.336469,-10.862575,-5.499141,-0.081304,-22.540702,-10.965084,-13.285669,-1.653736
3,pfizer03,D0,1,22.215,29.853,26.0340,4315.792,4523.093,4419.4425,36.477,...,4.808532,-67.201807,-25.932013,-10.461341,1.089484,0.636182,-17.882878,-14.317886,-6.529133,-3.052361
4,pfizer04,D0,1,17.951,23.404,20.6775,972.941,1039.918,1006.4295,23.440,...,10.327609,-71.728128,-16.488504,-3.533596,-3.646481,-2.752426,-20.316026,-6.476440,-5.125715,5.697594
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,pfizer57,D42,9,215554.461,224977.465,220265.9630,6559.721,6096.448,6328.0845,186.007,...,99.813571,91.810914,99.710513,96.322237,99.663171,98.506102,95.947713,99.766737,99.704093,99.848034
309,pfizer58,D0,9,54.800,57.220,56.0100,2018.522,2037.909,2028.2155,50.843,...,1.395875,-36.233175,-2.671736,-0.159208,-0.164996,-7.442042,-19.109093,4.861639,7.782129,-2.103302
310,pfizer58,D7,9,57.220,50.926,54.0730,2090.358,2115.281,2102.8195,44.541,...,19.162172,-1.432594,13.262700,12.931888,18.802561,14.343946,13.432648,16.714440,19.046663,17.758508
311,pfizer58,D21,9,4247.902,4379.121,4313.5115,1816.664,1853.637,1835.1505,29.923,...,18.268883,-17.419456,15.834752,11.776376,18.018992,13.704999,4.231354,20.106839,22.986129,21.518551


# Reshape vaccine data

In [4]:
vaccine_df.columns

Index(['study_id', 'tp', 'plate', 'CoV2_S_IgG', 'CoV2_S_IgG.1',
       'CoV2_S_IgG_Mean', 'NL63_S_IgG', 'NL63_S_IgG.1', 'NL63_S_IgG_Mean',
       'CoV2_N_IgG',
       ...
       'P9_RBD_ACE_B.1.429_percent', 'P9_RBD_ACE_B.1.351_percent',
       'P9_RBD_ACE_B.1.243_percent', 'P9_RBD_ACE_P.1_percent',
       'P9_RBD_ACE_B.1.526.2_percent', 'P9_RBD_ACE_B.1.1.7_percent',
       'P9_RBD_ACE_B.1.1.7_E484_percent', 'P9_RBD_ACE_B.1.617_percent',
       'P9_RBD_ACE_B.1.214_percent', 'P9_CoV2 _RBD_ACE_percent'],
      dtype='object', length=210)

In [5]:
vaccine_df = pd.melt(
    vaccine_df, id_vars=["study_id", "tp", "plate"], var_name="measurement"
)
vaccine_df

,study_id,tp,plate,measurement,value
0,pfizer00,D0,1,CoV2_S_IgG,698.328
1,pfizer01,D0,1,CoV2_S_IgG,34.175
2,pfizer02,D0,1,CoV2_S_IgG,55.502
3,pfizer03,D0,1,CoV2_S_IgG,22.215
4,pfizer04,D0,1,CoV2_S_IgG,17.951
...,...,...,...,...,...
64786,pfizer57,D42,9,P9_CoV2 _RBD_ACE_percent,99.848034
64787,pfizer58,D0,9,P9_CoV2 _RBD_ACE_percent,-2.103302
64788,pfizer58,D7,9,P9_CoV2 _RBD_ACE_percent,17.758508
64789,pfizer58,D21,9,P9_CoV2 _RBD_ACE_percent,21.518551


In [6]:
vaccine_df.dtypes

study_id       object
tp             object
plate           int64
measurement    object
value          object
dtype: object

In [7]:
# there are some non-numeric values in "value"
vaccine_df["value"].apply(np.isreal).value_counts()

True     57910
False     6881
Name: value, dtype: int64

In [8]:
"#VALUE!" in vaccine_df["value"].values, "#DIV/0!" in vaccine_df["value"].values

(True, True)

In [9]:
# convert to float and switch non-numeric values to nan
vaccine_df["value"] = pd.to_numeric(vaccine_df["value"], errors="coerce")
vaccine_df["value"].isna().value_counts()

False    64741
True        50
Name: value, dtype: int64

In [10]:
vaccine_df["plate"].value_counts()

4    7659
5    7659
6    7659
7    7659
1    7452
2    7452
3    7452
8    6417
9    5382
Name: plate, dtype: int64

In [11]:
vaccine_df["measurement"].unique()

array(['CoV2_S_IgG', 'CoV2_S_IgG.1', 'CoV2_S_IgG_Mean', 'NL63_S_IgG',
       'NL63_S_IgG.1', 'NL63_S_IgG_Mean', 'CoV2_N_IgG', 'CoV2_N_IgG.1',
       'CoV2_N_IgG_Mean', 'CoV1_S_IgG', 'CoV1_S_IgG.1', 'CoV1_S_IgG_Mean',
       'CoV2_NTD_IgG', 'CoV2_NTD_IgG.1', 'CoV2_NTD_IgG_Mean',
       'HKU1_S_IgG', 'HKU1_S_IgG.1', 'HKU1_S_IgG_Mean', 'OC43_S_IgG',
       'OC43_S_IgG.1', 'OC43_S_IgG_Mean', 'h229E_S_IgG', 'h229E_S_IgG.1',
       'h229E_S_IgG_Mean', 'CoV2_RBD_IgG', 'CoV2_RBD_IgG.1',
       'CoV2_RBD_IgG_Mean', 'CoV2_S_IgM', 'CoV2_S_IgM.1',
       'CoV2_S_IgM_Mean', 'NL63_S_IgM', 'NL63_S_IgM.1', 'NL63_S_IgM_Mean',
       'CoV2_N_IgM', 'CoV2_N_IgM.1', 'CoV2_N_IgM_Mean', 'CoV1_S_IgM',
       'CoV1_S_IgM.1', 'CoV1_S_IgM_Mean', 'CoV2_NTD_IgM',
       'CoV2_NTD_IgM.1', 'CoV2_NTD_IgM_Mean', 'HKU1_S_IgM',
       'HKU1_S_IgM.1', 'HKU1_S_IgM_Mean', 'OC43_S_IgM', 'OC43_S_IgM.1',
       'OC43_S_IgM_Mean', 'h229E_S_IgM', 'h229E_S_IgM.1',
       'h229E_S_IgM_Mean', 'CoV2_RBD_IgM', 'CoV2_RBD_IgM.1',
    

In [12]:
vaccine_df = vaccine_df[
    (vaccine_df["measurement"].str.contains("IgG"))
    | (vaccine_df["measurement"].str.contains("IgA"))
    | (vaccine_df["measurement"].str.contains("IgM"))
]
vaccine_df

,study_id,tp,plate,measurement,value
0,pfizer00,D0,1,CoV2_S_IgG,698.3280
1,pfizer01,D0,1,CoV2_S_IgG,34.1750
2,pfizer02,D0,1,CoV2_S_IgG,55.5020
3,pfizer03,D0,1,CoV2_S_IgG,22.2150
4,pfizer04,D0,1,CoV2_S_IgG,17.9510
...,...,...,...,...,...
25348,pfizer57,D42,9,CoV2_RBD_IgA_Mean,1685.2345
25349,pfizer58,D0,9,CoV2_RBD_IgA_Mean,46.6320
25350,pfizer58,D7,9,CoV2_RBD_IgA_Mean,44.0780
25351,pfizer58,D21,9,CoV2_RBD_IgA_Mean,102.0070


In [13]:
vaccine_df = vaccine_df[
    vaccine_df["measurement"].str.lower().str.contains("mean")
].copy()
vaccine_df

,study_id,tp,plate,measurement,value
626,pfizer00,D0,1,CoV2_S_IgG_Mean,682.8730
627,pfizer01,D0,1,CoV2_S_IgG_Mean,34.2955
628,pfizer02,D0,1,CoV2_S_IgG_Mean,57.4550
629,pfizer03,D0,1,CoV2_S_IgG_Mean,26.0340
630,pfizer04,D0,1,CoV2_S_IgG_Mean,20.6775
...,...,...,...,...,...
25348,pfizer57,D42,9,CoV2_RBD_IgA_Mean,1685.2345
25349,pfizer58,D0,9,CoV2_RBD_IgA_Mean,46.6320
25350,pfizer58,D7,9,CoV2_RBD_IgA_Mean,44.0780
25351,pfizer58,D21,9,CoV2_RBD_IgA_Mean,102.0070


In [14]:
vaccine_df["measurement"].value_counts()

CoV2_NTD_IgA_Mean    313
h229E_S_IgG_Mean     313
CoV2_RBD_IgM_Mean    313
CoV2_S_IgG_Mean      313
NL63_S_IgG_Mean      313
CoV2_RBD_IgG_Mean    313
CoV2_S_IgA_Mean      313
CoV2_NTD_IgM_Mean    313
CoV1_S_IgG_Mean      313
OC43_S_IgM_Mean      313
CoV2_N_IgM_Mean      313
OC43_S_IgG_Mean      313
CoV1_S_IgA_Mean      313
CoV2_S_IgM_Mean      313
HKU1_S_IgA_Mean      313
CoV2_RBD_IgA_Mean    313
h229E_S_IgM_Mean     313
NL63_S_IgA_Mean      313
NL63_S_IgM_Mean      313
CoV2_N_IgG_Mean      313
CoV2_NTD_IgG_Mean    313
OC43_S_IgA_Mean      313
CoV2_N_IgA_Mean      313
HKU1_S_IgG_Mean      313
CoV1_S_IgM_Mean      313
h229E_S_IgA_Mean     313
HKU1_S_IgM_Mean      313
Name: measurement, dtype: int64

In [15]:
vaccine_df["measurement_original_column_name"] = vaccine_df["measurement"].copy()

In [16]:
vaccine_df["measurement"] = (
    vaccine_df["measurement"].str.replace("_Mean", "").str.replace("_mean", "")
)
vaccine_df["measurement"].value_counts()

NL63_S_IgM      313
h229E_S_IgM     313
HKU1_S_IgG      313
CoV2_NTD_IgA    313
CoV1_S_IgM      313
OC43_S_IgA      313
CoV1_S_IgG      313
CoV2_NTD_IgG    313
CoV2_N_IgG      313
CoV2_N_IgM      313
h229E_S_IgG     313
CoV2_NTD_IgM    313
CoV1_S_IgA      313
CoV2_RBD_IgA    313
CoV2_S_IgM      313
CoV2_RBD_IgM    313
CoV2_RBD_IgG    313
NL63_S_IgA      313
HKU1_S_IgM      313
CoV2_S_IgA      313
CoV2_S_IgG      313
NL63_S_IgG      313
h229E_S_IgA     313
OC43_S_IgM      313
HKU1_S_IgA      313
CoV2_N_IgA      313
OC43_S_IgG      313
Name: measurement, dtype: int64

In [17]:
# extract measurement column info
measurement_parts = (
    vaccine_df["measurement"]
    .str.split("_", expand=True)
    .rename(columns={0: "virus", 1: "target", 2: "antibody"})
    .assign(variant_plate_type="Wuhan")
    .apply(lambda col: col.str.strip())
)
measurement_parts

,virus,target,antibody,variant_plate_type
626,CoV2,S,IgG,Wuhan
627,CoV2,S,IgG,Wuhan
628,CoV2,S,IgG,Wuhan
629,CoV2,S,IgG,Wuhan
630,CoV2,S,IgG,Wuhan
...,...,...,...,...
25348,CoV2,RBD,IgA,Wuhan
25349,CoV2,RBD,IgA,Wuhan
25350,CoV2,RBD,IgA,Wuhan
25351,CoV2,RBD,IgA,Wuhan


In [18]:
measurement_parts["virus"] = measurement_parts["virus"].replace({"CoV2": "Wuhan"})

In [19]:
measurement_parts["virus"].value_counts()

Wuhan    3756
h229E     939
NL63      939
OC43      939
CoV1      939
HKU1      939
Name: virus, dtype: int64

In [20]:
measurement_parts["target"].value_counts()

S      5634
N       939
RBD     939
NTD     939
Name: target, dtype: int64

In [21]:
measurement_parts["antibody"].value_counts()

IgM    2817
IgA    2817
IgG    2817
Name: antibody, dtype: int64

In [22]:
vaccine_df = pd.concat([vaccine_df, measurement_parts], axis=1)
vaccine_df

,study_id,tp,plate,measurement,value,measurement_original_column_name,virus,target,antibody,variant_plate_type
626,pfizer00,D0,1,CoV2_S_IgG,682.8730,CoV2_S_IgG_Mean,Wuhan,S,IgG,Wuhan
627,pfizer01,D0,1,CoV2_S_IgG,34.2955,CoV2_S_IgG_Mean,Wuhan,S,IgG,Wuhan
628,pfizer02,D0,1,CoV2_S_IgG,57.4550,CoV2_S_IgG_Mean,Wuhan,S,IgG,Wuhan
629,pfizer03,D0,1,CoV2_S_IgG,26.0340,CoV2_S_IgG_Mean,Wuhan,S,IgG,Wuhan
630,pfizer04,D0,1,CoV2_S_IgG,20.6775,CoV2_S_IgG_Mean,Wuhan,S,IgG,Wuhan
...,...,...,...,...,...,...,...,...,...,...
25348,pfizer57,D42,9,CoV2_RBD_IgA,1685.2345,CoV2_RBD_IgA_Mean,Wuhan,RBD,IgA,Wuhan
25349,pfizer58,D0,9,CoV2_RBD_IgA,46.6320,CoV2_RBD_IgA_Mean,Wuhan,RBD,IgA,Wuhan
25350,pfizer58,D7,9,CoV2_RBD_IgA,44.0780,CoV2_RBD_IgA_Mean,Wuhan,RBD,IgA,Wuhan
25351,pfizer58,D21,9,CoV2_RBD_IgA,102.0070,CoV2_RBD_IgA_Mean,Wuhan,RBD,IgA,Wuhan


In [23]:
vaccine_df["tp"].value_counts()

D21    1593
D0     1566
D42    1485
D28    1404
D7     1296
D90    1107
Name: tp, dtype: int64

In [24]:
vaccine_df["plate"].value_counts()

4    999
5    999
6    999
7    999
1    972
2    972
3    972
8    837
9    702
Name: plate, dtype: int64

In [25]:
def process_vaccine_timepoint(df_partial, timepoint):
    # at a given time point: only one measurement per patient-virus-target-antibody combo (i.e. not repeated across plates)
    assert all(
        df_partial.groupby(
            ["study_id", "virus", "target", "antibody", "variant_plate_type"]
        ).size()
        == 1
    )

    # patients are distributed across plates - each patient found on a single plate
    assert all(df_partial.groupby("study_id")["plate"].nunique() == 1)

    # patients are distributed across plates
    # combine measurements across plates
    vaccine_df_pivot = pd.pivot(
        df_partial,
        index="study_id",
        columns=[
            "virus",
            "target",
            "variant_plate_type",
            "antibody",
            "measurement_original_column_name",
        ],
        values="value",
    )

    ## set column names
    variable_info = vaccine_df_pivot.columns.to_frame().reset_index(drop=True)
    # create combined name
    variable_info["timepoint"] = timepoint
    variable_info["combined_name"] = variable_info[
        variable_info.columns.drop("measurement_original_column_name")
    ].apply("_".join, axis=1)
    variable_info = variable_info.set_index("combined_name")

    # set var names
    vaccine_df_pivot.columns = variable_info.index.copy()

    # drop patients with any NaNs
    vaccine_df_pivot = vaccine_df_pivot.dropna(how="any")
    assert not vaccine_df_pivot.isna().any().any()

    return vaccine_df_pivot, variable_info

In [26]:
vaccine_df["tp"].value_counts()

D21    1593
D0     1566
D42    1485
D28    1404
D7     1296
D90    1107
Name: tp, dtype: int64

In [27]:
# timepoint label map
map_vaccine_to_global_timepoint_labels = {
    "D0": "day 0 / pre-pandemic",
    "D7": "day 7 / week 1",
    "D21": "day 21 / weeks 2&3",
    "D28": "day 28 / week 4",
    "D42": "day 42 / weeks 5&6",
    "D90": "week 7 and later",
}
assert all(
    tp in map_vaccine_to_global_timepoint_labels.keys()
    for tp in vaccine_df["tp"].unique()
)

In [28]:
X_partial = []
var_partial = []
for vaccine_timepoint in vaccine_df["tp"].unique():
    associated_global_timepoint_label = map_vaccine_to_global_timepoint_labels[
        vaccine_timepoint
    ]
    print(vaccine_timepoint, "->", associated_global_timepoint_label)
    df_partial = vaccine_df[vaccine_df["tp"] == vaccine_timepoint]
    vaccine_df_pivot, variable_info = process_vaccine_timepoint(
        df_partial, associated_global_timepoint_label
    )
    X_partial.append(vaccine_df_pivot)
    var_partial.append(variable_info)
vaccine_df_pivot = pd.concat(X_partial, axis=1)
variable_info = pd.concat(var_partial, axis=0)

D0 -> day 0 / pre-pandemic
D7 -> day 7 / week 1
D42 -> day 42 / weeks 5&6
D21 -> day 21 / weeks 2&3
D28 -> day 28 / week 4
D90 -> week 7 and later


In [29]:
# note: there are NaNs - patients don't have entries for all timepoints
vaccine_df_pivot

combined_name,Wuhan_S_Wuhan_IgG_day 0 / pre-pandemic,NL63_S_Wuhan_IgG_day 0 / pre-pandemic,Wuhan_N_Wuhan_IgG_day 0 / pre-pandemic,CoV1_S_Wuhan_IgG_day 0 / pre-pandemic,Wuhan_NTD_Wuhan_IgG_day 0 / pre-pandemic,HKU1_S_Wuhan_IgG_day 0 / pre-pandemic,OC43_S_Wuhan_IgG_day 0 / pre-pandemic,h229E_S_Wuhan_IgG_day 0 / pre-pandemic,Wuhan_RBD_Wuhan_IgG_day 0 / pre-pandemic,Wuhan_S_Wuhan_IgM_day 0 / pre-pandemic,...,Wuhan_RBD_Wuhan_IgM_week 7 and later,Wuhan_S_Wuhan_IgA_week 7 and later,NL63_S_Wuhan_IgA_week 7 and later,Wuhan_N_Wuhan_IgA_week 7 and later,CoV1_S_Wuhan_IgA_week 7 and later,Wuhan_NTD_Wuhan_IgA_week 7 and later,HKU1_S_Wuhan_IgA_week 7 and later,OC43_S_Wuhan_IgA_week 7 and later,h229E_S_Wuhan_IgA_week 7 and later,Wuhan_RBD_Wuhan_IgA_week 7 and later
study_id,,,,,,,,,,,,,,,,,,,,,
pfizer00,682.8730,6715.5525,868.6600,1047.2245,83.2890,23255.8560,76538.4185,67044.9865,1635.2120,125.7255,...,738.2930,2451.9795,3575.1140,1482.2745,315.2020,688.6205,3176.8125,4639.2735,6983.9330,1921.2650
pfizer01,34.2955,701.3870,109.6080,41.2405,2.7410,3047.0230,14337.6380,22816.1735,101.3445,27.3405,...,305.6500,1436.2765,137.8840,104.4405,134.1925,272.7040,527.7500,3669.4570,2794.1980,744.8780
pfizer02,57.4550,1479.3955,94.4080,30.1870,2.3670,2754.0340,18616.2670,16538.5145,93.8270,136.5535,...,603.3530,5036.0035,655.2465,104.8235,412.0530,1872.2105,3525.6060,9635.7725,2061.1630,1967.1180
pfizer03,26.0340,4419.4425,37.7095,104.0490,4.2295,4661.5620,8355.9325,28847.0875,142.7325,132.9435,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pfizer04,20.6775,1006.4295,37.9500,75.3080,9.5560,69115.7905,65751.5645,45806.1560,283.4220,59.1320,...,118.7620,2879.6940,1607.3360,114.1405,144.7630,127.1325,4280.2025,3307.3135,2278.9975,390.6285
pfizer05,1123.2840,2847.1370,45.6835,1199.9800,11.0710,40578.4860,23135.9930,101683.9995,778.7100,165.7595,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pfizer06,19.6410,1669.3330,44.8760,80.3855,4.3340,4743.6040,24943.2335,16381.7135,96.1865,164.9535,...,215.7210,434.9470,2418.7435,281.2550,61.9825,135.1640,450.7205,676.9410,3883.9040,343.7970
pfizer07,159.3835,617.9735,41.6910,43.8285,5.3290,7970.1065,18396.3235,25714.9760,209.2095,400.7290,...,1450.2070,3079.0375,4169.0345,88.7890,476.7685,893.7950,1490.6010,2691.8405,13482.8515,1819.3630
pfizer08,59.6205,5423.1190,2240.0555,95.1185,1.1830,20453.3065,57279.4350,5553.3350,45.0800,50.7455,...,693.9680,408.1270,1073.6115,89.7510,47.5930,164.6755,2953.0230,4467.5960,18444.9745,327.7410


In [30]:
variable_info

,virus,target,variant_plate_type,antibody,measurement_original_column_name,timepoint
combined_name,,,,,,
Wuhan_S_Wuhan_IgG_day 0 / pre-pandemic,Wuhan,S,Wuhan,IgG,CoV2_S_IgG_Mean,day 0 / pre-pandemic
NL63_S_Wuhan_IgG_day 0 / pre-pandemic,NL63,S,Wuhan,IgG,NL63_S_IgG_Mean,day 0 / pre-pandemic
Wuhan_N_Wuhan_IgG_day 0 / pre-pandemic,Wuhan,N,Wuhan,IgG,CoV2_N_IgG_Mean,day 0 / pre-pandemic
CoV1_S_Wuhan_IgG_day 0 / pre-pandemic,CoV1,S,Wuhan,IgG,CoV1_S_IgG_Mean,day 0 / pre-pandemic
Wuhan_NTD_Wuhan_IgG_day 0 / pre-pandemic,Wuhan,NTD,Wuhan,IgG,CoV2_NTD_IgG_Mean,day 0 / pre-pandemic
...,...,...,...,...,...,...
Wuhan_NTD_Wuhan_IgA_week 7 and later,Wuhan,NTD,Wuhan,IgA,CoV2_NTD_IgA_Mean,week 7 and later
HKU1_S_Wuhan_IgA_week 7 and later,HKU1,S,Wuhan,IgA,HKU1_S_IgA_Mean,week 7 and later
OC43_S_Wuhan_IgA_week 7 and later,OC43,S,Wuhan,IgA,OC43_S_IgA_Mean,week 7 and later


In [31]:
# attach status
vaccine_participants = vaccine_participants.set_index("PID")
vaccine_participants["Status"] = "Vaccinee"
# reorder
vaccine_participants = vaccine_participants.loc[vaccine_df_pivot.index]
vaccine_participants

,COVID Positive Ever?,Status
study_id,,
pfizer00,No,Vaccinee
pfizer01,No,Vaccinee
pfizer02,No,Vaccinee
pfizer03,No,Vaccinee
pfizer04,No,Vaccinee
pfizer05,No,Vaccinee
pfizer06,No,Vaccinee
pfizer07,No,Vaccinee
pfizer08,No,Vaccinee


In [32]:
# anndata requires string indices
vaccine_participants.index = vaccine_participants.index.astype(str)
vaccine_df_pivot.index = vaccine_df_pivot.index.astype(str)

In [33]:
adata_vaccine = anndata.AnnData(
    X=vaccine_df_pivot, obs=vaccine_participants, var=variable_info
)
adata_vaccine

AnnData object with n_obs × n_vars = 59 × 162
    obs: 'COVID Positive Ever?', 'Status'
    var: 'virus', 'target', 'variant_plate_type', 'antibody', 'measurement_original_column_name', 'timepoint'

In [34]:
adata_vaccine.var

,virus,target,variant_plate_type,antibody,measurement_original_column_name,timepoint
combined_name,,,,,,
Wuhan_S_Wuhan_IgG_day 0 / pre-pandemic,Wuhan,S,Wuhan,IgG,CoV2_S_IgG_Mean,day 0 / pre-pandemic
NL63_S_Wuhan_IgG_day 0 / pre-pandemic,NL63,S,Wuhan,IgG,NL63_S_IgG_Mean,day 0 / pre-pandemic
Wuhan_N_Wuhan_IgG_day 0 / pre-pandemic,Wuhan,N,Wuhan,IgG,CoV2_N_IgG_Mean,day 0 / pre-pandemic
CoV1_S_Wuhan_IgG_day 0 / pre-pandemic,CoV1,S,Wuhan,IgG,CoV1_S_IgG_Mean,day 0 / pre-pandemic
Wuhan_NTD_Wuhan_IgG_day 0 / pre-pandemic,Wuhan,NTD,Wuhan,IgG,CoV2_NTD_IgG_Mean,day 0 / pre-pandemic
...,...,...,...,...,...,...
Wuhan_NTD_Wuhan_IgA_week 7 and later,Wuhan,NTD,Wuhan,IgA,CoV2_NTD_IgA_Mean,week 7 and later
HKU1_S_Wuhan_IgA_week 7 and later,HKU1,S,Wuhan,IgA,HKU1_S_IgA_Mean,week 7 and later
OC43_S_Wuhan_IgA_week 7 and later,OC43,S,Wuhan,IgA,OC43_S_IgA_Mean,week 7 and later


In [35]:
adata_vaccine.obs["Status"].value_counts()

Vaccinee    59
Name: Status, dtype: int64

In [36]:
adata_vaccine.write(
    f"{config.paths.generated_data_dir}/partial.pfizer_vaccine.coronavirus_plate.original.h5"
)

... storing 'COVID Positive Ever?' as categorical


... storing 'Status' as categorical


... storing 'virus' as categorical


... storing 'target' as categorical


... storing 'variant_plate_type' as categorical


... storing 'antibody' as categorical


... storing 'measurement_original_column_name' as categorical


... storing 'timepoint' as categorical
